In [49]:
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'       #Disactivate multiprocessing for numpy
import numpy as np
import matplotlib.pyplot as plt
import json
import gymnasium as gym
import yaml
from datetime import datetime

import stable_baselines3
from stable_baselines3.common.env_checker import check_env
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize, SubprocVecEnv
from stable_baselines3 import PPO, A2C, SAC, TD3
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise
from stable_baselines3.common.callbacks import EvalCallback, CallbackList, CheckpointCallback, StopTrainingOnNoModelImprovement

from sogym.mmc_optim import run_mmc
from sogym.env import sogym
from sogym.expert_generation import generate_expert_dataset, generate_mmc_solutions, generate_dataset
from sogym.utils import profile_and_analyze,ImageDictExtractor, CustomBoxDense
from sogym.callbacks import FigureRecorderCallback, MaxRewardCallback, GradientNormCallback, GradientClippingCallback
from sogym.pretraining import pretrain_agent, ExpertDataSet

import torch
import torch as th
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split, Dataset
from IPython.display import display

%load_ext autoreload
%autoreload 2

print('SB3 version:', stable_baselines3.__version__)
# Let's make the code device agnostic:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
SB3 version: 2.2.1
Using device: cuda


---
### Environment test and visualization:

In [50]:
# Let's test the environment using the check_env util from SB3:
observation_type = 'topopt_game'
train_env = sogym(mode='train',observation_type=observation_type,vol_constraint_type='hard',resolution=50,check_connectivity = True)
eval_env = sogym(mode='test',observation_type=observation_type,vol_constraint_type='hard',resolution=50,check_connectivity=False)

In [51]:
import matplotlib.pyplot as plt

reward = 0.0
while reward == 0.0:
    obs, info = train_env.reset()
    dones = False
    while not dones:
        action = train_env.action_space.sample()
        obs, reward, dones, truncated, info = train_env.step(action)

    fig = train_env.plot()
fig.savefig('env_test.png')

KeyboardInterrupt: 

In [ ]:
# It will check your custom environment and output additional warnings if needed
check_env(sogym(mode='train',observation_type='topopt_game'))

In [ ]:
# Example usage
# Specify the number of episodes to run
num_episodes = 20
# Call the profile_and_analyze function
result_df = profile_and_analyze(num_episodes, train_env)
# Print the resulting DataFrame
result_df

In [ ]:
obs = train_env.reset()
cfg = {
            'optimizer':'mma', #optimiser choice
            'xInt':0.25, #initial interval of components in x
            'yInt':0.25, #initial interval of components in y
            'E':1.0, #Young's modulus
            'nu':0.3, #Poisson ratio
            'h':1, #thickness
            'dgt0':5, #significant digit of sens.
            'scl':1, #scale factor for obj
            'p':6,  #power of super ellipsoid
            'lmd':100, #power of KS aggregation   
            'maxiter':500, # maximum number of outer iterations
            'alpha':1e-9, # This is the threshold level in the Heaviside function
            'epsilon':0.2, #This is the regularization term in the Heaviside function
            'maxinnerinit':1, # This is the maximum number of inner iterations for GCMMA
            'switch':-0.000002, # This is the switch criteria for the hybrid optimizer
            'convergence_threshold':2e-4, #This is the threshold for the relative change in the objective function
            'xmin':(0.0, 0.0, 0.0, 0.00, 0.00, -np.pi),
            'xmax':(train_env.dx, train_env.dy, 0.7*min(train_env.dx,train_env.dy), 0.05*min(train_env.dx,train_env.dy),0.05*min(train_env.dx,train_env.dy), np.pi)
        }

#run_mmc(train_env.conditions,train_env.nelx,train_env.nely,train_env.dx,train_env.dy,plotting='contour',verbose=0,cfg=cfg)
dataset_folder = "/home/thomas/Documents/scratch_thomas/GitHub/sogym_v2/dataset/topologies/mmc"
#generate_mmc_solutions(key=0,dataset_folder="/home/thomas/Documents/scratch_thomas/GitHub/sogym_v2/dataset/topologies/mmc")
generate_dataset(dataset_folder= dataset_folder, num_threads=32, num_samples=20000)

In [ ]:
# Let's visualize the training environment on a random problem statement and visualize a 'successful' solution:
reward = 0.0
while reward==0.0:
    obs = train_env.reset()
    done = False
    while not done:
        action = train_env.action_space.sample()
        obs, reward, done,truncated, info = train_env.step(action)
        
# print("Volume: ", train_env.volume)
print("Reward ",reward)

train_env.plot()

In [ ]:
# Create a figure and axes for the subplots
fig, axes = plt.subplots(nrows=5, ncols=5, figsize=(10, 10))
axes = axes.flatten()

# Initialize the index for the current subplot
subplot_index = 0

# Let's visualize the training environment on a random problem statement and visualize a 'successful' solution:
reward = 0.0
while reward == 0.0:
    obs = train_env.reset()
    done = False
    while not done:
        action = train_env.action_space.sample()
        obs, reward, done, truncated, info = train_env.step(action)
        
        # Plot the current observation image
        axes[subplot_index].imshow(obs['strain_energy'].T, cmap='gray')
        axes[subplot_index].axis('off')
        axes[subplot_index].set_title(f"Timestep {subplot_index+1}")
        
        # Increment the subplot index
        subplot_index += 1
        
        # If all subplots are filled, display the plot and reset the index
        if subplot_index == len(axes):
            plt.tight_layout()
            plt.show()
            subplot_index = 0

# Print the reward
print("Reward:", reward)

# Plot the final state of the training environment
train_env.plot()

# Display any remaining subplots
if subplot_index > 0:
    for i in range(subplot_index, len(axes)):
        axes[i].axis('off')
    plt.tight_layout()
    plt.show()


In [3]:
# Specify the number of permutations to generate
num_permutations = None
observation_type = "topopt_game"

# Specify the environment configuration (optional)
env_kwargs = {
    'mode': 'train',
    'observation_type': observation_type,
    'vol_constraint_type': 'hard',
    'seed': 42,
    'resolution' : 50, 
    'check_connectivity':True
}

directory_path = "/home/thomas/Documents/scratch_thomas/GitHub/sogym_v2/dataset/topologies_narval"
expert_observations, expert_actions = generate_expert_dataset(directory_path,env_kwargs, plot_terminated=False,num_permutations = num_permutations, file_fraction=1.0)
# Save the dataset
import pickle

# Save the data using pickle
with open('expert_dataset_narval_topoptgame.pkl', 'wb') as f:
    pickle.dump({'expert_observations': expert_observations, 'expert_actions': expert_actions}, f, protocol=4)
print(len(expert_observations))

Processing files: 100%|██████████| 38855/38855 [3:42:43<00:00,  2.91file/s]  


7


In [52]:
import pickle

# Load the data using pickle
with open('expert_dataset_narval_topoptgame.pkl', 'rb') as f:
    data = pickle.load(f)

expert_observations = data['expert_observations']
expert_actions = data['expert_actions']

In [56]:
\
# Assuming you have the expert_dataset defined
expert_dataset = ExpertDataSet(expert_observations, expert_actions, train_env)
# Get a random sample from the dataset
sample_idx = np.random.randint(len(expert_dataset))
sample = expert_dataset[sample_idx]

# Extract the observation and reward from the sample
observation, action = sample

# Subplot with image, strain_energy, and structure_strain_energy observations:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Plot image observation
axes[0].imshow(observation['image'].T, cmap='gray', origin='lower')
axes[0].axis('on')
axes[0].set_title("Image Observation")

# Plot strain_energy observation
axes[1].imshow(observation['structure_strain_energy'].T, origin='lower')
axes[1].axis('on')
axes[1].set_title("Structure Strain Energy Observation")

print(action)
plt.tight_layout()
plt.savefig('expert_observation.png', dpi=300)
plt.show()

tensor([ 0.0581, -0.9009,  0.8737,  0.7976, -1.3026, -1.3027],
       dtype=torch.float64)


In [57]:
obs = train_env.reset()

#use action and plot the result
obs, rewards, dones,truncated, info = train_env.step(np.array(action))

In [58]:
plt.imshow(obs['image'].T,origin='lower')
plt.savefig('expert_action.png', dpi=300)

In [59]:
chosen_policy = "MlpPolicy" if observation_type == 'box_dense' else "MultiInputPolicy"

feature_extractor = ImageDictExtractor if observation_type == 'image' or observation_type == 'topopt_game' else CustomBoxDense

# Load the YAML file
env=train_env

with open("algorithms.yaml", "r") as file:
    config = yaml.safe_load(file)

# Extract the parameters for the desired algorithm
algorithm_name = "PPO"  # or "TD3"
algorithm_params = config[algorithm_name]

policy_kwargs = dict(
    features_extractor_class=feature_extractor,
    net_arch = config['common']['net_arch'],
    share_features_extractor = False
)

# Create the model based on the algorithm name and parameters
if algorithm_name == "SAC":
    model = SAC(env=env,
                policy = chosen_policy, 
                policy_kwargs=policy_kwargs,
                device=device, 
                **algorithm_params)

elif algorithm_name == "PPO":
    model = PPO(env=env, 
                policy = chosen_policy, 
                policy_kwargs=policy_kwargs,
                device = device, 
                **algorithm_params)

elif algorithm_name == "TD3":
    # Create the action noise object
    n_actions = env.action_space.shape[-1]
    action_noise_params = algorithm_params.pop("action_noise")
    action_noise = NormalActionNoise(mean=action_noise_params["mean"] * np.ones(n_actions),
                                     sigma=action_noise_params["sigma"] * np.ones(n_actions))
    model = TD3(env=env,
                policy =chosen_policy, 
                policy_kwargs=policy_kwargs,
                action_noise=action_noise,
                device=device, 
                **algorithm_params)

# Get the current date and time
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the tb_log_name string
tb_log_name = f"{algorithm_name}_{current_datetime}"

/home/thomas/anaconda3/envs/SB3_update/lib/python3.10/site-packages/stable_baselines3/ppo/ppo.py:155: UserWarning: You have specified a mini-batch size of 4096, but because the `RolloutBuffer` is of size `n_steps * n_envs = 64`, after every 0 untruncated mini-batches, there will be a truncated mini-batch of size 64
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=64 and n_envs=1)
  warnings.warn(


In [35]:
from torchinfo import summary
total_params = sum(p.numel() for p in model.policy.parameters())
print(f"Total number of parameters: {total_params:,}")
data = {k: v for k, v in observation.items()}
# Assuming you have a PyTorch model named 'model' and the input size is (3, 224, 224)
summary(model.policy)


Total number of parameters: 85,996,045


Layer (type:depth-idx)                   Param #
MultiInputActorCriticPolicy              6
├─ImageDictExtractor: 1-1                --
│    └─ReLU: 2-1                         --
│    └─ModuleDict: 2-2                   --
│    │    └─Sequential: 3-1              22,784
│    │    └─Sequential: 3-2              93,248
│    │    └─Sequential: 3-3              16,768
│    │    └─Sequential: 3-4              16,768
│    │    └─Sequential: 3-5              93,248
│    │    └─Sequential: 3-6              16,768
├─ImageDictExtractor: 1-2                (recursive)
│    └─ReLU: 2-3                         --
│    └─ModuleDict: 2-4                   (recursive)
│    │    └─Sequential: 3-7              (recursive)
│    │    └─Sequential: 3-8              (recursive)
│    │    └─Sequential: 3-9              (recursive)
│    │    └─Sequential: 3-10             (recursive)
│    │    └─Sequential: 3-11             (recursive)
│    │    └─Sequential: 3-12             (recursive)
├─ImageDictExtractor

In [60]:
pretrain_agent(
    model,
    expert_observations,
    expert_actions,
    env,
    test_env = eval_env,
    batch_size=4096,
    epochs=500,
    scheduler_gamma=0.98,
    learning_rate=1.0,
    log_interval=5,
    no_cuda=False,
    seed=1,
    verbose=True,
    test_batch_size=512,
    early_stopping_patience=300,
    plot_curves=True,
    tensorboard_log_dir="tb_logs/imitation",
    save_path="checkpoints/imitation_PPO",
    comet_ml_api_key="No20MKxPKu7vWLOUQCFBRO8mo",
    comet_ml_project_name="pretraining_rl",
    comet_ml_experiment_name="PPO_1",
    eval_freq = 5,
    l2_reg_strength=0.001,
)

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorboard, torch.
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml Experiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/thomasrb/pretraining-rl/97a45188a1b74315b4f5973f5585e0a2
COMET INFO:   Metrics [count] (min, max):
COMET INFO:     grad_norm [8]  : (9.999997824963602, 10.000000433511623)
COMET INFO:     mae [8]        : (0.6573815548862346, 1.7594083989552685)
COMET INFO:     mean_reward    : 0.042885462567210195
COMET INFO:     std_reward     : 0.0011408701136307441
COMET INFO:     test_loss [8]  : (0.8659590560230338, 5.786423800384693)
COMET INFO:     test_mae [8]   : (0.6573815548862346, 1.7594083989552685)
COMET INFO:    

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: tensorboard, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/thomasrb/pretraining-rl/8169cc7a7c2b431e99d3e4ef9db25df7



Train Epoch: 1 [4096/248672 (2%)]	Loss: 2.518659	Grad Norm: 4.628726	LR: 1.000000
Train Epoch: 1 [24576/248672 (10%)]	Loss: 2.919998	Grad Norm: 9.999999	LR: 1.000000
Train Epoch: 1 [45056/248672 (18%)]	Loss: 2.779126	Grad Norm: 9.999999	LR: 1.000000
Train Epoch: 1 [65536/248672 (26%)]	Loss: 3.073122	Grad Norm: 9.999998	LR: 1.000000
Train Epoch: 1 [86016/248672 (34%)]	Loss: 3.257378	Grad Norm: 9.999999	LR: 1.000000
Train Epoch: 1 [106496/248672 (43%)]	Loss: 2.565800	Grad Norm: 8.192236	LR: 1.000000
Train Epoch: 1 [126976/248672 (51%)]	Loss: 2.866419	Grad Norm: 9.999999	LR: 1.000000


KeyboardInterrupt: 

---
### Multiprocessing

In [ ]:
#from transformers import AutoTokenizer, AutoModel
from stable_baselines3.common.vec_env import DummyVecEnv, VecCheckNan
import multiprocessing

#tokenizer = AutoTokenizer.from_pretrained("huggingface/CodeBERTa-small-v1")
#model = AutoModel.from_pretrained("huggingface/CodeBERTa-small-v1").to('cuda')

# Set number of cpus to use automatically:
num_cpu = multiprocessing.cpu_count()
print("Using {} cpus!".format(num_cpu))
observation_type = "topopt_game"

train_env = sogym(mode='train',observation_type=observation_type,vol_constraint_type = 'hard',resolution=50,check_connectivity=True)#,model=model,tokenizer=tokenizer)
env= make_vec_env(lambda:train_env, n_envs=num_cpu,vec_env_cls=SubprocVecEnv)
env = VecCheckNan(env, raise_exception=True)
#env=VecNormalize(env,gamma=1.0)

eval_env = sogym(mode='test',observation_type=observation_type,vol_constraint_type='hard',resolution=50,check_connectivity=True)#,model=model,tokenizer=tokenizer)
eval_env = make_vec_env(lambda:eval_env, n_envs=1,vec_env_cls=SubprocVecEnv)
#eval_env =VecNormalize(eval_env,gamma=1.0)


--- 
### Defining the model

In [ ]:
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for TD3
n_actions = env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.5 * np.ones(n_actions))

chosen_policy = "MlpPolicy" if observation_type == 'box_dense' else "MultiInputPolicy"

feature_extractor = ImageDictExtractor if observation_type == 'image' or observation_type=="topopt_game" else CustomBoxDense

# Load the YAML file

with open("algorithms.yaml", "r") as file:
    config = yaml.safe_load(file)

# Extract the parameters for the desired algorithm
algorithm_name = "PPO"  # or "TD3"
algorithm_params = config[algorithm_name]

policy_kwargs = dict(
    features_extractor_class=feature_extractor,
    net_arch = config['common']['net_arch'],
    share_features_extractor = False,
)
# Create the model based on the algorithm name and parameters
if algorithm_name == "SAC":
    model = SAC(env=env,
                policy = chosen_policy, 
                policy_kwargs=policy_kwargs,
                #action_noise = action_noise,
                ent_coef = 0.0,
                device=device, 
                **algorithm_params)

elif algorithm_name == "PPO":
    model = PPO(env=env, 
                policy = chosen_policy, 
                policy_kwargs=policy_kwargs,
                device = device, 
                **algorithm_params)

elif algorithm_name == "TD3":
    # Create the action noise object
    n_actions = env.action_space.shape[-1]
    action_noise_params = algorithm_params.pop("action_noise")
    action_noise = NormalActionNoise(mean=action_noise_params["mean"] * np.ones(n_actions),
                                     
                                     sigma=action_noise_params["sigma"] * np.ones(n_actions))
    model = TD3(env=env,
                policy =chosen_policy, 
                policy_kwargs=policy_kwargs,
                action_noise=action_noise,
                device=device, 
                **algorithm_params)

# Get the current date and time
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")

# Create the tb_log_name string
tb_log_name = f"{algorithm_name}_{current_datetime}"

In [ ]:
# Save a checkpoint every 1000 steps
checkpoint_callback = CheckpointCallback(
  save_freq=50000//num_cpu,
  save_path="./checkpoints/",
  name_prefix=tb_log_name,
  save_replay_buffer=True,
  save_vecnormalize=True,
)

eval_callback = EvalCallback(eval_env,
                             log_path='tb_logs',
                             eval_freq=5000//num_cpu,
                             deterministic=True,
                            n_eval_episodes=10,
                             render=False,
                             best_model_save_path='./checkpoints',
                             verbose=0)

callback_list = CallbackList([eval_callback,
                         checkpoint_callback,
                         MaxRewardCallback(verbose=1),
                         GradientClippingCallback(clip_value=1.0, verbose=1),
                         GradientNormCallback(verbose=1),
                         FigureRecorderCallback(check_freq=5000//num_cpu,eval_env=eval_env),
                         StopTrainingOnNoModelImprovement(max_no_improvement_evals=50, min_evals=100, verbose=1)
                         ])

--- 
### Training

Save the model:

If model is on-policy:
#model.save("sac_pendulum")
#loaded_model = SAC.load("sac_pendulum")

if model is off-policy, we also need to save the replay buffer:
#model.save_replay_buffer("sac_replay_buffer")
#loaded_model.load_replay_buffer("sac_replay_buffer")

If the environment is normalized:
#env.save('env_saved.pkl')
#env = VecNormalize.load('env_saved.pkl',env)


In [ ]:

import torch.nn as nn

def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_uniform_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
            
#model = SAC.load("checkpoints/imitation_SAC",env =env) 
model.set_parameters("checkpoints/imitation_PPO.zip")
model.set_parameters("imitation_PPO_critic")

train_critic_only = False
if train_critic_only:
    #Freeze everything:
    for name, param in model.policy.named_parameters():
        if param.requires_grad:
            param.requires_grad=False

    if algorithm_name =='SAC'
        # Unfreeze critic:
        for param in model.policy.critic.parameters():
            if param.requires_grad==False:
                param.requires_grad=True

        for param in model.policy.critic_target.parameters():
            if param.requires_grad==False:
                param.requires_grad=True
        
        #Reset critic networks:
        if hasattr(model.policy.critic_target, 'reset_parameters'):
            print(' resetting')
            model.policy.critic_target.reset_parameters()
        else:
            model.policy.critic_target.apply(init_weights)

            
        if hasattr(model.policy.critic, 'reset_parameters'):
            print('resetting')
            model.policy.critic_target.reset_parameters() 
        else:
            model.policy.critic.apply(init_weights)


    if algorithm_name == 'PPO':
        for param in model.policy.mlp_extractor.value_net.parameters():
            if param.requires_grad==False:
                param.requires_grad=True
            
        for param in model.policy.value_net.parameters():
            if param.requires_grad==False:
                param.requires_grad=True
        

        if hasattr(model.policy.value_net, 'reset_parameters'):
            print(' resetting')
            model.policy.value_net.reset_parameters()
        else:
            model.policy.value_net.apply(init_weights)
            
        if hasattr(model.policy.mlp_extractor.value_net, 'reset_parameters'):
            print(' resetting')
            model.policy.mlp_extractor.value_net.reset_parameters() 
        else:
            model.policy.mlp_extractor.value_net.apply(init_weights)

In [ ]:

#print(model.batch_size)
#model.load_replay_buffer("sac_replay_buffer")
model.learn(20000000,
            callback=callback_list, 
            tb_log_name=tb_log_name
            )
#model.save('model_saved_march15',)
#model.save_replay_buffer("sac_replay_buffer_march15")

#env.save('env_saved.pkl')

In [ ]:
model.save('checkpoints/imitation_PPO_critic')

---
### Let's visualize the agent's performance:

In [ ]:
env=sogym(mode='train',observation_type='image',vol_constraint_type='hard' ,resolution = 50)
#env= make_vec_env(lambda:env, n_envs=1,vec_env_cls=SubprocVecEnv)
env

In [46]:
obs,info=env.reset()
dones=False
saved_conditions = env.conditions
saved_nelx, saved_nely = env.nelx, env.nely
saved_dx, saved_dy = env.dx, env.dy
#use deepcopy to save 
while dones== False:
    action, _states = model.predict(obs,deterministic=True)
    print(action)
    obs, rewards, dones,truncated, info = env.step(action)
print("Desired volume:",saved_conditions['volfrac'],"Obtained volume:",env.volume)
print("Env reward:",rewards)
fig = env.plot()

[-0.99356264 -0.88954514  0.9281974  -0.01584718  0.8976205   1.        ]
[-1.         -0.16608484  0.29554093 -0.54767656  0.5325971   0.55634016]
[-1.          0.15776648  0.8151289   0.12417474  0.9366684   0.6389875 ]
[-0.82835186  0.5015193   0.96246237 -0.62295455  0.96702075  0.96023256]
[-1.         -1.          1.         -0.14496084  0.9393588   1.        ]
[-0.9223096  -0.3790284   1.         -0.35445184  1.          1.        ]
[-1.         -0.2980384   1.          0.00239816  0.6195617   0.7502269 ]
[-0.9283385  -0.21001871  0.93565124  0.03575581  0.91625756  1.        ]
Desired volume: 0.49 Obtained volume: 0.3380188606665687
Env reward: 0.0


In [47]:
fig.savefig('trained_agent.png', dpi=300)

In [ ]:
xval, f0val,it, H, Phimax, allPhi, den, N, cfg = run_mmc(saved_conditions,saved_nelx,saved_nely,saved_dx,saved_dy,plotting='contour')